<a href="https://colab.research.google.com/github/GitData-GA/shot-marilyns-analysis/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries and scripts

In [ ]:
import os
import shutil
import sys

!git clone https://github.com/GitData-GA/shot-marilyns-analysis.git
sys.path.insert(0, './shot-marilyns-analysis/src')

import sma
os.makedirs(r'img')

Cloning into 'shot-marilyns-analysis'...
remote: Enumerating objects: 1044, done.
remote: Counting objects: 100% (331/331), done.
remote: Compressing objects: 100% (177/177), done.
remote: Total 1044 (delta 92), reused 271 (delta 82), pack-reused 713
Receiving objects: 100% (1044/1044), 86.53 MiB | 10.49 MiB/s, done.
Resolving deltas: 100% (309/309), done.


# Improt images and prepare data

In [ ]:
img_links = {
    "orange_marilyn": "https://shotmarilyns.gd.edu.kg/assets/img/1_1_orange_marilyn.jpg",
    "red_marilyn": "https://shotmarilyns.gd.edu.kg/assets/img/1_2_red_marilyn.jpg",
    "turq_marilyn": "https://shotmarilyns.gd.edu.kg/assets/img/1_3_turq_marilyn.jpg",
    "blue_marilyn": "https://shotmarilyns.gd.edu.kg/assets/img/1_4_blue_marilyn.jpg",
    "eggblue_marilyn": "https://shotmarilyns.gd.edu.kg/assets/img/1_5_eggblue_marilyn.jpg"
}

## Save images to local directory and show them

In [ ]:
sma.utils.save_img(img_links, img_idx=1, show_img=False)

## Store the images as a NumPy array

In [ ]:
np_img = sma.utils.np_convert(img_links)
np_img['orange_marilyn']

## Store the images as a dictionary of 5 Pandas dataframes with HEX codes

In [ ]:
pd_img = sma.utils.pd_convert(img_links)
pd_img['orange_marilyn']

# Generating plots

## Distribution plots

In [ ]:
sma.plot.distribution(np_img, img_idx=2, show_plot=True)

## Relative conditional entropy plots

In [ ]:
sma.plot.entropy_heatmap(np_img, img_idx=3, show_plot=True)

## RGB space scatterplots

In [ ]:
sma.plot.scatter(pd_img, img_idx=4, show_plot=True)

## KMeans clustering

In [ ]:
kmean_result = sma.cluster.kmeans(pd_img, n_clusters=15)

### Scatter plot by clusters

In [ ]:
sma.plot.scatter(pd_img, img_idx=5, kmeans=kmean_result, show_plot=True)

### Bar chart by clusters

In [ ]:
sma.plot.bar(pd_img, img_idx=6, kmeans=kmean_result, show_plot=True)

### Color ribbon by clusters

In [ ]:
sma.plot.ribbon(pd_img, img_idx=7, kmeans=kmean_result, show_plot=True)

# Save all images in a zip file

In [ ]:
shutil.make_archive("img.zip".replace('.zip', ''), 'zip', 'img')

# Model for face segmentation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
import tensorflow as tf
from tensorflow.keras import layers, models

# Reshape the images
images = {}
for key, value in np_img.items():
    images[key] = value.reshape((960, 960, 3))

# Convert to numpy arrays
X = np.array([images[key] for key in images.keys()])

# Normalize the images
X = X / 255.0

# Generate synthetic masks
def generate_synthetic_mask(image, threshold=0.5):
    gray_image = np.mean(image, axis=-1)
    mask = gray_image > threshold
    return mask.astype(np.uint8)

Y = np.array([generate_synthetic_mask(image) for image in X])

# Reshape Y to have a single channel
Y = Y.reshape((Y.shape[0], Y.shape[1], Y.shape[2], 1))

In [ ]:
def unet_model(input_size=(960, 960, 3)):
    inputs = layers.Input(input_size)

    # Encoder
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    p3 = layers.MaxPooling2D((2, 2))(c3)

    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c4)
    p4 = layers.MaxPooling2D((2, 2))(c4)

    c5 = layers.Conv2D(1024, (3, 3), activation='relu', padding='same')(p4)
    c5 = layers.Conv2D(1024, (3, 3), activation='relu', padding='same')(c5)

    # Decoder
    u6 = layers.Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = layers.concatenate([u6, c4])
    c6 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(u6)
    c6 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c6)

    u7 = layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = layers.concatenate([u7, c3])
    c7 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(u7)
    c7 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c7)

    u8 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = layers.concatenate([u8, c2])
    c8 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u8)
    c8 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c8)

    u9 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = layers.concatenate([u9, c1])
    c9 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u9)
    c9 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c9)

    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(c9)

    model = models.Model(inputs=[inputs], outputs=[outputs])
    return model


In [ ]:
# K-Fold Cross Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

fold_no = 1
val_losses = []
val_accuracies = []

for train_index, val_index in kf.split(X):
    # Split data
    X_train, X_val = X[train_index], X[val_index]
    Y_train, Y_val = Y[train_index], Y[val_index]

    # Build model
    model = unet_model()
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Train model
    history = model.fit(X_train, Y_train,
                        epochs=50,
                        batch_size=2,
                        validation_data=(X_val, Y_val))

    # Evaluate model
    loss, accuracy = model.evaluate(X_val, Y_val)
    val_losses.append(loss)
    val_accuracies.append(accuracy)

    print(f"Fold {fold_no}: Validation Loss: {loss}, Validation Accuracy: {accuracy}")
    fold_no += 1

print(f"Average Validation Loss: {np.mean(val_losses)}, Average Validation Accuracy: {np.mean(val_accuracies)}")


In [ ]:
def preprocess_image(image):
    image = image / 255.0
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    return image

def postprocess_mask(mask, threshold=0.5):
    mask = mask.squeeze()  # Remove batch dimension
    mask = mask > threshold  # Apply threshold
    return mask.astype(np.uint8)

# Example: Predict on a new image
new_image = images['turq_marilyn']  # Replace with new image as needed
preprocessed_image = preprocess_image(new_image)
predicted_mask = model.predict(preprocessed_image)
postprocessed_mask = postprocess_mask(predicted_mask)


In [ ]:
import cv2

def overlay_mask_on_image(image, mask, color=(0, 255, 0), alpha=0.5):
    """Overlay the mask on the image with a given color and transparency."""
    overlay = image.copy()
    for c in range(3):
        overlay[:, :, c] = np.where(mask == 1, color[c], overlay[:, :, c])
    return cv2.addWeighted(overlay, alpha, image, 1 - alpha, 0)

# Overlay the mask on the original image
colored_mask = overlay_mask_on_image(new_image, postprocessed_mask)


In [ ]:
import matplotlib.pyplot as plt

def display_images(original, mask, overlay):
    plt.figure(figsize=(15, 5))

    plt.subplot(1, 3, 1)
    plt.title('Original Image')
    plt.imshow(original)
    plt.axis('off')

    plt.subplot(1, 3, 2)
    plt.title('Predicted Mask')
    plt.imshow(mask)
    plt.axis('off')

    plt.subplot(1, 3, 3)
    plt.title('Overlay')
    plt.imshow(overlay)
    plt.axis('off')

    plt.show()

# Display the original image, the mask, and the overlay
display_images(new_image, postprocessed_mask, colored_mask)
